In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
import os
from datetime import datetime
import time
import plotly

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, auc
import plotly.figure_factory as ff
from sklearn.preprocessing import OneHotEncoder
from data_set_params import DataSetParams
from scipy.io import wavfile
params = DataSetParams()

In [3]:
#import os
#
#def get_size(start_path = '.'):
#    total_size = 0
#    for dirpath, dirnames, filenames in os.walk(start_path):
#        for f in filenames:
#            fp = os.path.join(dirpath, f)
#            # skip if it is symbolic link
#            if not os.path.islink(fp):
#                print(fp)
#                print("\tsize:", os.path.getsize(fp))
#                total_size += os.path.getsize(fp)
#
#    return total_size
#total_size = get_size()

In [4]:
#print(total_size, 'bytes')
#print(total_size/(1024), 'kilobytes')
#print(total_size/(1024**2), 'megabytes')
#print(total_size/(1024**3), 'gigabytes')

In [5]:
#!pip install tensorflow

In [6]:
#!zip -r models_20220303.zip models

In [7]:
#!zip -r results.zip results

In [8]:
#! unzip ~/data/wav_clips.zip -d ~/data/

In [9]:
#! unzip ~/data/ff1010bird.zip -d ~/data/

In [10]:
#import matplotlib.pyplot as plt

In [11]:
ff1010bird_meta = pd.read_csv('C:\\Users\\Anthony\\Downloads\\ff1010bird_metadata.csv')

In [12]:
ff1010bird_meta

itemid  hasbird
0      64486        0
1       2525        0
2      44981        0
3     101323        0
4     165746        0
...      ...      ...
7685  168059        0
7686  164922        0
7687   80789        1
7688  104733        1
7689   40565        0

[7690 rows x 2 columns]

In [13]:
b_dict = {0: 'not_bird',
          1: 'bird'}

In [14]:
#os.mkdir(os.path.join(data_dir, 'bird'))
#os.mkdir(os.path.join(data_dir, 'not_bird'))

In [15]:
#os.rename(data_dir, "path/to/new/destination/for/file.foo")
#os.listdir(data_dir)

In [16]:
#for i,row in ff1010bird_meta.iterrows():
#    st_dir  = os.path.join(data_dir, str(row.itemid)+'.wav')
#    sav_dir = os.path.join(data_dir, b_dict[row.hasbird], str(row.itemid)+'.wav')
    #print(st_dir)
    #print()
#    os.rename(st_dir, sav_dir)
    #print(row.itemid)
    #print(row.hasbird)
    #print(str(row.itemid)+'.wav' in os.listdir(data_dir))
    #print()

In [17]:
data_dir   = 'C:\\Users\\Anthony\\Downloads\\ff1010bird\\ff1010bird_wav\\wav'

In [18]:
#data_dir = pathlib.Path('/root/tensorflow_datasets/downloads/extracted/TAR_GZ.opihi.cs.uvic.ca_sound_music_speechbya81rFcWfLSW6ey5cynqyeq2qiePcL-7asMoNO6IQ0.tar.gz/music_speech')
#data_dir   = 'C:\\Users\\Anthony\\Downloads\\ff1010bird\\ff1010bird_wav\\wav'#'richfield_birds_split'#'dublin_dl_birds_split'#
categories = np.array(tf.io.gfile.listdir(data_dir))
categories = [category for category in categories if 'wav' not in category and '.TAG' not in category]
categories

['bird', 'not_bird']

In [19]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)

    # Note: You'll use indexing here instead of tuple unpacking to enable this 
    # to work in a TensorFlow graph.
    return parts[-2]

In [20]:
def preload_vgg19(input_shape):
    model = models.Sequential([
        layers.Input(shape=input_shape),
        Flatten(), # Flatten dimensions to for use in FC layers
        Dense(256, activation='relu'),
        Dropout(0.5), # Dropout layer to reduce overfitting
        Dense(64, activation='relu'),
        Dense(num_classes, activation='softmax') # Softmax for multiclass
    ])
    model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return model

In [21]:
def preload_vgg19_concat(input_shape, num_channels):
    from tensorflow.keras.layers import concatenate
    out_list   = []
    input_list = []
    for c in range(num_channels):
        input_a = layers.Input(shape=input_shape)
        input_list.append(input_a)
        #out_list.append(vgg_model.output)
    #vgg_model.summary()
    #x = vgg_model.output
    concatenated = concatenate(input_list, axis=-1)
    x = Flatten()(concatenated) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    concat_vgg_model = Model(inputs=input_list, outputs=x)
    concat_vgg_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return concat_vgg_model

In [22]:
filenames = tf.io.gfile.glob(str(data_dir) + '/*/*')
#filenames = tf.io.gfile.glob('birds/*/*')
filenames = [filename for filename in filenames if 'wav' in filename]
filenames = tf.random.shuffle(filenames)

In [23]:
AUTOTUNE   = tf.data.experimental.AUTOTUNE
batch_size = 32
EPOCHS     = 50

In [24]:
num_classes = len(categories)

In [25]:
from tf_helpers import *

In [26]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score, top_k_accuracy_score
from sklearn.preprocessing import OneHotEncoder
from math import prod

In [27]:
def save_results(model, x_test, y_true, name, filename_run, index):
        #model.save('models/'+filename_run+'.h5')
    
        pred_lists = model.predict(x_test)
        y_pred     = np.argmax(pred_lists, axis=-1)
        pred_df    = pd.DataFrame(pred_lists, columns = categories)
        
        softmax_prediction_df = pred_df.apply(lambda x: np.exp(x - np.max(x))/np.exp(x - np.max(x)).sum(), axis=1)
        softmax_prediction_df.to_csv('results/'+filename_run+'_softmax_prediction_df.csv')
        
        cm = confusion_matrix(y_true, y_pred)
        fig = plotly_cm(cm, categories)
        fig.write_html('results/'+filename_run+'_confusion_matrix.html')
        
        num_trainable    = sum([prod(w.shape) for w in model.trainable_weights])
        num_nontrainable = sum([prod(w.shape) for w in model.non_trainable_weights])
        
        onehot_data = OneHotEncoder(sparse=False)
        onehot_data = onehot_data.fit_transform(np.array(y_true).reshape(len(y_true),1))
        roc_auc = [0]*num_classes
        
        for i in range(num_classes):
            roc_auc[i] = roc_auc_score(onehot_data[:, i], softmax_prediction_df.to_numpy()[:, i])
        
        name_df = pd.DataFrame(data={
                  'model':     name}, index=[index])
        metric_df = pd.DataFrame(data={
                  'top_1_acc': [accuracy_score(y_pred, y_true)],
                  #'top_5_acc': [top_k_accuracy_score(y_true, softmax_prediction_df, k=5)],
                  'precision': [precision_score(y_pred, y_true, average = 'weighted')], 
                  'f1':        [f1_score(y_pred, y_true, average = 'weighted')]
                 })
        param_df = pd.DataFrame(data={
                  'trainable_params': [num_trainable],
                  'nontrainable_params': [num_nontrainable]
                 })
        auc_df = pd.DataFrame([roc_auc], columns = ['auc_'+categories[i].replace(' ', '') for i in range(num_classes)])
        
        metric_df =  pd.concat([name_df, metric_df],axis=1)
        metric_df.to_csv('results/'+filename_run+'_metric_df.csv')
        
        param_df  =  pd.concat([name_df, param_df],axis=1)
        param_df.to_csv('results/'+filename_run+'_param_df.csv')
        
        auc_df    =  pd.concat([name_df, auc_df],axis=1)
        auc_df.to_csv('results/'+filename_run+'_auc_df.csv')

In [28]:
def load_vgg19(input_shape):
    vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)
    vgg_model.trainable = False ## Not trainable weights
    #vgg_model.summary()
    x = vgg_model.output
    x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    transfer_vgg_model = Model(inputs=vgg_model.input, outputs=x)
    transfer_vgg_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return transfer_vgg_model

In [29]:
def load_vgg19_concat(input_shape, num_channels):
    from tensorflow.keras.layers import concatenate
    out_list   = []
    input_list = []
    for c in range(num_channels):
        vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
        vgg_model._name = vgg_model._name + str(c)
        for layer in vgg_model.layers:
            layer._name = layer._name + '_' + str(c)
        vgg_model.trainable = False ## Not trainable weights
        input_a = vgg_model.input
        input_list.append(input_a)
        out_list.append(vgg_model.output)
    #vgg_model.summary()
    #x = vgg_model.output
    concatenated = concatenate(out_list)
    x = Flatten()(concatenated) # Flatten dimensions to for use in FC layers
    x = Dense(256*num_channels, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64*num_channels, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    concat_vgg_model = Model(inputs=input_list, outputs=x)
    concat_vgg_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return concat_vgg_model

In [30]:
def load_resnet50(input_shape):
    resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    resnet_model.trainable = False ## Not trainable weights
    #resnet_model.summary()
    x = resnet_model.output
    x = Flatten()(x) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    transfer_resnet_model = Model(inputs=resnet_model.input, outputs=x)
    transfer_resnet_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return transfer_resnet_model

In [31]:
def load_resnet50_concat(input_shape, num_channels):
    from tensorflow.keras.layers import concatenate
    out_list   = []
    input_list = []
    for c in range(num_channels):
        resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
        resnet_model._name = resnet_model._name + str(c)
        for layer in resnet_model.layers:
            layer._name = layer._name + '_' + str(c)
        resnet_model.trainable = False ## Not trainable weights
        input_a = resnet_model.input
        input_list.append(input_a)
        out_list.append(resnet_model.output)
    #vgg_model.summary()
    #x = vgg_model.output
    concatenated = concatenate(out_list)
    x = Flatten()(concatenated) # Flatten dimensions to for use in FC layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x) # Dropout layer to reduce overfitting
    x = Dense(64, activation='relu')(x)
    x = Dense(num_classes, activation='softmax')(x) # Softmax for multiclass
    concat_resnet_model = Model(inputs=input_list, outputs=x)
    concat_resnet_model.compile(
        optimizer = optimizers.Adam(learning_rate=0.0001),
        loss      = losses.SparseCategoricalCrossentropy(),
        metrics   = 'accuracy'
        )
    return concat_resnet_model

In [32]:
def main_model_run(filenames, index,
                   vgg19        = False,
                   vgg19_concat = False,
                   resnet50     = False,
                   resnet50_concat = False,
                   smallcnn = False,
                   concat   = False,
                   concat2  = False,
                   concat3  = False,
                   concat4  = False):
    print("Index: ", index)
    
    filenames   = tf.random.shuffle(filenames)
    all_labs    = [get_label(y).numpy().decode() for y in filenames]
    filename_df = pd.DataFrame({'name': filenames.numpy(),
                                'label': all_labs})
    
    train, rest_df = train_test_split(filename_df, test_size=0.2, stratify=filename_df[['label']])
    val, test      = train_test_split(rest_df, test_size=0.5, stratify = rest_df[['label']])
    
    train_files = tf.random.shuffle(train['name'])
    val_files   = tf.random.shuffle(val['name'])
    test_files  = tf.random.shuffle(test['name'])

    def concat_xy(ds):
            x_tmp  = [x for x,_ in ds]
            x_tmp  = tf.stack(x_tmp)
            xs_tmp = tf.unstack(x_tmp, axis=-1)
            xs_tmp = [tf.expand_dims(x_ind, axis=-1) for x_ind in xs_tmp]
            y      = np.array([y for _,y in ds])
            return xs_tmp, y
    
    print('Getting data')
    choices  = ['Mod']
    train_ds = preprocess_dataset(train_files, choices, categories, req_width=750, resize = 4, spec_norm = True)
    val_ds   = preprocess_dataset(val_files,   choices, categories, req_width=750, resize = 4, spec_norm = True)
    test_ds  = preprocess_dataset(test_files,  choices, categories, req_width=750, resize = 4, spec_norm = True)
    
    train_ds_vgg = preprocess_dataset(train_files, choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    val_ds_vgg   = preprocess_dataset(val_files,   choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    test_ds_vgg  = preprocess_dataset(test_files,  choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    
    X_train_vgg = np.array([x for x,y in train_ds_vgg])
    X_val_vgg   = np.array([x for x,y in val_ds_vgg])
    X_test_vgg  = np.array([x for x,y in test_ds_vgg])
    
    choices = ['Mod', 'AbsRe', 'AbsIm'] #['Mod','AbsRe', 'AbsIm', 'LogMod', 'Ang']
    train_ds_mult = preprocess_dataset(train_files, choices, categories, req_width=750, resize = 4, spec_norm = True)
    val_ds_mult   = preprocess_dataset(val_files,   choices, categories, req_width=750, resize = 4, spec_norm = True)
    test_ds_mult  = preprocess_dataset(test_files,  choices, categories, req_width=750, resize = 4, spec_norm = True)
    
    X_train_mult, y_train = concat_xy(train_ds_mult)
    X_val_mult,   y_val   = concat_xy(val_ds_mult)
    X_test_mult,  y_test  = concat_xy(test_ds_mult)
    
    num_channels = len(choices)
    nc = num_channels
    
    train_ds_mult_vgg = preprocess_dataset(train_files, choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    val_ds_mult_vgg   = preprocess_dataset(val_files,   choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    test_ds_mult_vgg  = preprocess_dataset(test_files,  choices, categories, req_width=750, single_to_rgb = True, resize = 4, spec_norm = True)
    X_train_mult_vgg, _ = concat_xy(train_ds_mult_vgg)
    X_val_mult_vgg,   _ = concat_xy(val_ds_mult_vgg)
    X_test_mult_vgg,  _ = concat_xy(test_ds_mult_vgg)
    
    X_train_mult_vgg = np.array(X_train_mult_vgg)
    X_val_mult_vgg   = np.array(X_val_mult_vgg)
    X_test_mult_vgg  = np.array(X_test_mult_vgg)
    
    X_train_mult_vgg = [tf.concat(X_train_mult_vgg[[idx, idx+nc, idx+2*nc]], axis=-1) for idx in range(nc)]
    X_val_mult_vgg   = [tf.concat(X_val_mult_vgg[[idx, idx+nc, idx+2*nc]],   axis=-1) for idx in range(nc)]
    X_test_mult_vgg  = [tf.concat(X_test_mult_vgg[[idx, idx+nc, idx+2*nc]],  axis=-1) for idx in range(nc)]
    
    X_train_mult_vgg = [np.rollaxis(np.squeeze(xt), 0, 4) for xt in X_train_mult_vgg]
    X_val_mult_vgg   = [np.rollaxis(np.squeeze(xt), 0, 4) for xt in X_val_mult_vgg]
    X_test_mult_vgg  = [np.rollaxis(np.squeeze(xt), 0, 4) for xt in X_test_mult_vgg]

    print("Done")
    
    
    concat_shape     = X_train_mult[0].shape[1:]
    vgg_concat_shape = X_train_mult_vgg[0].shape[1:]
    
    for spec, _ in train_ds.take(1):
        input_shape = spec.shape
    for spec, _ in train_ds_vgg.take(1):
        input_shape_vgg = spec.shape
        #print(tf.math.reduce_std(spec[:,:,0]))
        #print(tf.math.reduce_std(spec[:,:,1]))
        #print(tf.math.reduce_std(spec[:,:,2]))
        
    train_ds = train_ds.batch(batch_size)
    val_ds   = val_ds.batch(batch_size)
    test_ds  = test_ds.batch(batch_size)
    train_ds = train_ds.cache().prefetch(AUTOTUNE)
    val_ds   = val_ds.cache().prefetch(AUTOTUNE)
    test_ds  = test_ds.cache().prefetch(AUTOTUNE)
    
    if data_dir.startswith("C:"):
        sv_dir = '_'.join(data_dir.split(os.path.sep)[1:-2])
    else:
        sv_dir = data_dir
    filename_idx = datetime.now().strftime("%Y%m%d-%H%M%S").replace('-', '_')+'_'+sv_dir.replace('/','_')+'_'+str(index)
    results_folder = 'results'
    if not os.path.isdir(results_folder):
        os.mkdir(results_folder)
    
    train.to_csv(results_folder+'/'+filename_idx+'_filenames_train.csv')
    val.to_csv(results_folder+'/'+filename_idx+'_filenames_val.csv')
    test.to_csv(results_folder+'/'+filename_idx+'_filenames_test.csv')
    #np.save('results/'+filename_idx+'_filenames.npy', filenames.numpy())
    
    #this will save the model performing best on val accuracy
    def best_model_cp():
        checkpoint = tf.keras.callbacks.ModelCheckpoint(
            "best_model",
            monitor = "val_accuracy",
            mode    = "max",
            save_best_only = True,
            save_weights_only = True)
        return checkpoint
    
    ## Load and run models
    
    if vgg19 or vgg19_concat:
        vgg_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape_vgg)
    #VGG19
    if vgg19:
        print("VGG19")
        tmp_vgg_trainX = vgg_model.predict(X_train_vgg)
        tmp_vgg_valX   = vgg_model.predict(X_val_vgg)
        tmp_vgg_testX  = vgg_model.predict(X_test_vgg)
        model = preload_vgg19(tmp_vgg_trainX.shape[1:])
        model_name   = 'vgg19'
        filename_run = filename_idx+'_'+model_name
        print(model.summary())
        history = model.fit(tmp_vgg_trainX, y_train,
                            validation_data = (tmp_vgg_valX, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, tmp_vgg_testX, y_test, model_name, filename_run, index)
    
    
    #VGG19_concat
    if vgg19_concat:
        print("VGG19_concat")
        tmp_X_train_mult_vgg = [vgg_model.predict(xt) for xt in X_train_mult_vgg]
        tmp_X_val_mult_vgg   = [vgg_model.predict(xt) for xt in X_val_mult_vgg]
        tmp_X_test_mult_vgg  = [vgg_model.predict(xt) for xt in X_test_mult_vgg]
        #model    = load_vgg19_concat(vgg_concat_shape, num_channels)
        model_name   = 'vgg19_concat'
        filename_run = filename_idx+'_'+model_name
        model = preload_vgg19_concat(tmp_X_train_mult_vgg[0].shape[1:], num_channels)
        print(model.summary())
        history = model.fit(tmp_X_train_mult_vgg, y_train,
                            validation_data = (tmp_X_val_mult_vgg, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, tmp_X_test_mult_vgg, y_test, model_name, filename_run, index)


    #ResNet50
    if resnet50:
        print("ResNet50")
        model = load_resnet50(input_shape_vgg)
        model_name   = 'resnet50'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(train_ds,
                            validation_data = val_ds,
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, test_ds, y_test, model_name, filename_run, index)
    
    #ResNet50_concat
    if resnet50_concat:
        print("ResNet50_concat")
        model    = load_resnet50_concat(vgg_concat_shape, num_channels)
        model_name   = 'resnet50_concat'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(X_train_vgg, y_train,
                            validation_data = (X_val_vgg, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, X_test_vgg, y_test, model_name, filename_run, index)

    #small_cnn
    if smallcnn:
        print("Small CNN")
        model  = main_cnn(input_shape, num_classes)
        model_name   = 'smallcnn'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(train_ds,
                            validation_data = val_ds,
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, test_ds, y_test, model_name, filename_run, index)
    
    #concat
    if concat:
        print("Concat")
        model        = concat_model(concat_shape, num_channels, num_classes)
        model_name   = 'concat'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(X_train_mult, y_train,
                            validation_data = (X_val_mult, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            batch_size      = batch_size,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, X_test_mult, y_test, model_name, filename_run, index)

    #concat2
    if concat2:
        print("Concat2")
        model     = concat_model2(concat_shape, num_channels, num_classes)
        model_name   = 'concat2'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(X_train_mult, y_train,
                            validation_data = (X_val_mult, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            batch_size      = batch_size,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, X_test_mult, y_test, model_name, filename_run, index)
    
    #concat3
    if concat3:
        print("Concat3")
        model     = concat_model3(concat_shape, num_channels, num_classes)
        model_name   = 'concat3'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(X_train_mult, y_train,
                            validation_data = (X_val_mult, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            batch_size      = batch_size,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, X_test_mult, y_test, model_name, filename_run, index)
    
    #concat4
    if concat4:
        print("Concat4")
        model     = concat_model4(concat_shape, num_channels, num_classes)
        model_name   = 'concat4'
        filename_run = filename_idx+'_'+model_name

        history = model.fit(X_train_mult, y_train,
                            validation_data = (X_val_mult, y_val),
                            callbacks       = [best_model_cp()],
                            epochs          = EPOCHS,
                            batch_size      = batch_size,
                            verbose         = 1)

        pd.DataFrame(history.history).to_csv(results_folder+'/'+filename_run+'_model_history.csv')

        model.load_weights("best_model")

        save_results(model, X_test_mult, y_test, model_name, filename_run, index)

## Evaluate

In [33]:
#audio_df = pd.DataFrame({'fname': [], 'sr': [], 'len': []})

In [34]:
#for file_path in filenames:
#    audio_binary  = tf.io.read_file(file_path)
#    waveform, sr  = decode_audio(audio_binary)
#    sr = sr.numpy()
#    if len(waveform)/sr < 0.23:
#        os.remove(file_path.numpy().decode())
#        continue
#    row = pd.DataFrame({'fname': [file_path], 'sr': [sr], 'len': [len(waveform)/sr]})
#    audio_df = pd.concat([audio_df, row])

In [35]:
#audio_binary  = tf.io.read_file(filenames[0])
#waveform, sr  = decode_audio(audio_binary)

In [36]:
#audio_df.reset_index(inplace=True)

In [37]:
#audio_df.iloc[8028]['fname'].numpy().decode()

In [38]:
#audio_df[audio_df['len']<0.23]

In [39]:
len(filenames)

7690

In [40]:
tic = time.time()

In [41]:
for i in range(1,5+1):
    main_model_run(filenames[:1000], i,
                   vgg19        = True,
                   vgg19_concat = True
                   #resnet50     = True,
                   #resnet50_concat = True,
                   #smallcnn = True,
                   #concat   = True
                  )
                   #concat2  = True,
                   #concat3  = True,
                   #concat4  = True)
    print("Time so far:", time.time() - tic)

Index:  1
Getting data
Done
VGG19
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 47104)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               12058880  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130       
Total params: 12,075,458
Trainable params: 12,075,458
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
25/25 [==============================] - 1s 36ms/step - loss

25/25 [==============================] - 4s 133ms/step - loss: 1.7936 - accuracy: 0.6725 - val_loss: 0.7551 - val_accuracy: 0.7700
Epoch 2/50
25/25 [==============================] - 3s 126ms/step - loss: 1.1398 - accuracy: 0.6775 - val_loss: 0.4479 - val_accuracy: 0.8300
Epoch 3/50
25/25 [==============================] - 3s 126ms/step - loss: 0.6467 - accuracy: 0.7538 - val_loss: 0.6765 - val_accuracy: 0.5300
Epoch 4/50
25/25 [==============================] - 3s 128ms/step - loss: 0.5266 - accuracy: 0.7650 - val_loss: 0.4349 - val_accuracy: 0.8200
Epoch 5/50
25/25 [==============================] - 3s 128ms/step - loss: 0.4735 - accuracy: 0.7925 - val_loss: 0.4262 - val_accuracy: 0.8600
Epoch 6/50
25/25 [==============================] - 3s 125ms/step - loss: 0.4564 - accuracy: 0.7987 - val_loss: 0.4454 - val_accuracy: 0.8300
Epoch 7/50
25/25 [==============================] - 3s 132ms/step - loss: 0.4534 - accuracy: 0.8062 - val_loss: 0.4205 - val_accuracy: 0.8000
Epoch 8/50
25/25 

KeyboardInterrupt: 

In [ ]:
toc = time.time()

In [ ]:
print(toc-tic)

In [ ]:
#print(res_df_t.to_latex(bold_rows = True))

In [ ]:
#!zip -r results.zip results